# This example demonstrate how to train YOLO detector model

In [1]:
import sys
import os
import json
import yaml
import shutil
import matplotlib.pyplot as plt

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

from NomeroffNet.tools.yolo_tools import convert_dataset_to_yolo_format

0it [00:00, ?it/s]

git clone https://github.com/ultralytics/yolov5.git


100%|██████████| 7044.0/7044.0 [00:01<00:00, 4043.68it/s]  

In [2]:
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:
%matplotlib inline 

# Download dataset and yolo repo

In [4]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

# get yolov5
info = modelhub.download_repo_for_model("yolov5")
repo_path = info["repo_path"]

# auto download latest dataset
info = modelhub.download_dataset_for_model("yolov5")
PATH_TO_DATASET = info["dataset_path"]

# Convert dataset to yolo format

In [5]:
CLASSES = ['numberplate']
STATES  = ["val","train"]
#STATES  = ["val"]

PATH_TO_RES_ANN = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/labels/{}")
PATH_TO_RES_IMAGES  = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/images/{}")

PATH_TO_JSON = os.path.join(PATH_TO_DATASET, "{}/via_region_data.json")
PATH_TO_IMAGES = os.path.join(PATH_TO_DATASET, "{}/")

In [6]:
EXIST_STRATEGY = "exist_ok" # "delete", "exist_ok"

for state in STATES:
    path_to_res_ann    = PATH_TO_RES_ANN.format(state)
    path_to_res_images = PATH_TO_RES_IMAGES.format(state)
    
    if os.path.exists(path_to_res_ann) and os.path.exists(path_to_res_images) and EXIST_STRATEGY == "exist_ok":
        print("[INFO]", state, "data exists")
        continue
    if EXIST_STRATEGY == "delete":
        shutil.rmtree(path_to_res_ann)
        shutil.rmtree(path_to_res_images)
    
    print("[INFO]", state, "data creating...")
    os.makedirs(path_to_res_ann, exist_ok=True)
    os.makedirs(path_to_res_images, exist_ok=True)
    
    path_to_json    = PATH_TO_JSON.format(state)
    path_to_images  = PATH_TO_IMAGES.format(state)
    
    convert_dataset_to_yolo_format(
        path_to_res_ann, 
        path_to_res_images,
        path_to_images, 
        path_to_json, 
        debug=False,
        is_generate_image_rotation_variants=True
    )
    

[INFO] val data exists
[INFO] train data exists


# Create yaml config

In [7]:
yaml_path = os.path.join(os.path.dirname(PATH_TO_DATASET), "./npdata/numberplate_config.yaml")
yaml_dataset_config = {
    # Train/val/test sets
    "path": os.path.dirname(PATH_TO_DATASET), 
    "train": "./npdata/images/train",
    "val": "./npdata/images/val",
    #"test": "" # test images (optional)
    
    # Classes
    "nc": 1,  # number of classes
    "names": [ "numberplate" ]  # class names
}
print("[INFO] config:", yaml_dataset_config)
print("[INFO] config saved into", yaml_path)
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_dataset_config, 
              yaml_file, 
              default_flow_style=False)

[INFO] config: {'path': '/var/www/nomeroff-net24/NomeroffNet/tools/../../data/./dataset/Detector/yolov5', 'train': './npdata/images/train', 'val': './npdata/images/val', 'nc': 1, 'names': ['numberplate']}
[INFO] config saved into /var/www/nomeroff-net24/NomeroffNet/tools/../../data/./dataset/Detector/yolov5/./npdata/numberplate_config.yaml


# CLI yolo training

In [8]:
!cd {repo_path}; \
 wandb offline; \
 python3.9 train.py \
    --img 640 \
    --batch 8 \
    --epochs 100 \
    --data {yaml_path} \
    --weights yolov5s.pt

/bin/bash: рядок 1: wandb: команду не знайдено
train: weights=yolov5s.pt, cfg=, data=/var/www/nomeroff-net24/NomeroffNet/tools/../../data/./dataset/Detector/yolov5/./npdata/numberplate_config.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-137-g2c63175 torch 1.10.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3090, 24265MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05

In [9]:
repo_path

'/mnt/store/nomeroff-net/nomeroff-net/NomeroffNet/tools/../../data/./repos/Detector/yolov5'

## Run wandb localy

```
docker run --rm -it -v wandb:{repo_path}/wandb -p 8080:8080 --ip 0.0.0.0 --name wandb-local wandb/local
```